In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
# Load the data with the first column as datetime and handle comma as decimal separator
file_path = 'LD2011_2014.txt'
data = pd.read_csv(
    file_path,
    sep=';',                        # Semicolon-separated
    header=0,                       # Use the first row as the header
    parse_dates=[0],                # Parse the first column as datetime
    index_col=0,                    # Set the first column as the index
    dayfirst=True,                  # Assume the date format is day-first
    decimal=','                     # Use comma as decimal separator
)

# Display the first few rows to confirm correct loading
#print(data.head())

In [ ]:
# Convert kW to kWh by dividing each value by 4
data_kwh = data / 4

# Display the first few rows to verify
#print(data_kwh.head())

In [ ]:
# Split the data by date to cover different seasons
train_data = data_kwh['2011-01-01':'2012-12-31']
val_data = data_kwh['2013-01-01':'2013-12-31']
test_data = data_kwh['2014-01-01':'2014-12-31']

# Print the shapes to verify
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

In [ ]:
import numpy as np
import pandas as pd

# Define a function to inject anomalies and label them
def inject_and_label_anomalies(data, anomaly_percentage=0.01, spike_factor=3, prolonged_anomaly_duration=4):
    """
    Inject synthetic anomalies by randomly selecting points and adding spikes, dips, or prolonged high consumption.
    
    Parameters:
    - data (pd.DataFrame): The dataset where anomalies are injected.
    - anomaly_percentage (float): The percentage of points to modify with anomalies.
    - spike_factor (float): The multiplier for spikes to simulate anomalies.
    - prolonged_anomaly_duration (int): Number of consecutive data points to create prolonged high consumption anomalies.
    
    Returns:
    - data_with_anomalies (pd.DataFrame): Dataset with synthetic anomalies and anomaly labels.
    """
    # Ensure the index is in datetime format
    data.index = pd.to_datetime(data.index)
    
    # Create a copy to avoid modifying the original data
    data_with_anomalies = data.copy()
    
    # Add a column for anomaly labels (0 = normal, 1 = anomaly)
    data_with_anomalies['anomaly'] = 0
    
    # Calculate the total number of anomalies to inject
    num_anomalies = int(len(data) * anomaly_percentage)
    
    # Inject short-term spikes and dips
    short_term_indices = np.random.choice(data.index, size=num_anomalies, replace=False)
    for idx in short_term_indices:
        col = np.random.choice(data.columns[:-1])  # Avoid 'anomaly' column if added
        if np.random.rand() > 0.5:
            # Spike: Increase value
            data_with_anomalies.loc[idx, col] *= spike_factor
        else:
            # Dip: Decrease value
            data_with_anomalies.loc[idx, col] /= spike_factor
        
        # Mark as anomaly
        data_with_anomalies.loc[idx, 'anomaly'] = 1
    
    # Inject prolonged high consumption anomalies
    prolonged_indices = np.random.choice(data.index[:-prolonged_anomaly_duration], size=num_anomalies // 2, replace=False)
    for idx in prolonged_indices:
        # Apply prolonged high consumption by setting a range of consecutive rows
        for i in range(prolonged_anomaly_duration):
            col = np.random.choice(data.columns[:-1])  # Avoid 'anomaly' column if added
            # Ensure idx + timedelta works by using the datetime index
            data_with_anomalies.loc[idx + pd.Timedelta(minutes=15 * i), col] *= spike_factor
            data_with_anomalies.loc[idx + pd.Timedelta(minutes=15 * i), 'anomaly'] = 1
    
    return data_with_anomalies

# Inject anomalies into validation and test sets and label them
val_data_with_anomalies = inject_and_label_anomalies(val_data, anomaly_percentage=0.01, spike_factor=3)
test_data_with_anomalies = inject_and_label_anomalies(test_data, anomaly_percentage=0.01, spike_factor=3)

# Display sample rows with anomalies for verification
print("Sample validation data with anomalies:")
#print(val_data_with_anomalies[val_data_with_anomalies['anomaly'] == 1].sample(5))

print("Sample test data with anomalies:")
#print(test_data_with_anomalies[test_data_with_anomalies['anomaly'] == 1].sample(5))

In [ ]:
test_data_with_anomalies['anomaly'].value_counts()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report

# Assuming `data_kwh` is the original DataFrame with values converted to kWh

# Step 1: Split the data by date and retain the 'anomaly' column if it exists
train_data = data_kwh['2011-01-01':'2012-12-31'].copy()
val_data_with_anomalies = data_kwh['2013-01-01':'2013-12-31'].copy()
test_data_with_anomalies = data_kwh['2014-01-01':'2014-12-31'].copy()

# Separate 'anomaly' columns for future evaluation if they exist
train_anomaly = train_data['anomaly'].copy() if 'anomaly' in train_data.columns else None
val_anomaly = val_data_with_anomalies['anomaly'].copy() if 'anomaly' in val_data_with_anomalies.columns else None

# Drop 'anomaly' column temporarily for model input
X_train = train_data.drop(columns=['anomaly'], errors='ignore')
X_val = val_data_with_anomalies.drop(columns=['anomaly'], errors='ignore')

# Step 2: Train the Isolation Forest on X_train
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Step 3: Predict anomalies on the validation set
# Store predictions in a new DataFrame to avoid overwriting val_data_with_anomalies
predictions = pd.DataFrame(index=val_data_with_anomalies.index)
predictions['predicted_anomaly'] = iso_forest.predict(X_val)
predictions['predicted_anomaly'] = predictions['predicted_anomaly'].apply(lambda x: 1 if x == -1 else 0)

# Re-attach original 'anomaly' column and predictions for evaluation
val_data_with_anomalies = val_data_with_anomalies.assign(anomaly=val_anomaly)
val_data_with_anomalies['predicted_anomaly'] = predictions['predicted_anomaly']

# Fill NaN values in 'anomaly' and 'predicted_anomaly' columns if they exist
val_data_with_anomalies['anomaly'] = val_data_with_anomalies['anomaly'].fillna(0)
val_data_with_anomalies['predicted_anomaly'] = val_data_with_anomalies['predicted_anomaly'].fillna(0)

# Display a sample of results to confirm NaNs are handled
print("Sample of validation data with anomalies and predicted anomalies after filling NaNs:")
print(val_data_with_anomalies[['anomaly', 'predicted_anomaly']].sample(10))

# Step 4: Evaluate Model Performance
if 'anomaly' in val_data_with_anomalies.columns:
    print("Classification Report for Validation Data:")
    print(classification_report(val_data_with_anomalies['anomaly'], val_data_with_anomalies['predicted_anomaly']))
else:
    print("'anomaly' column missing in validation data. Cannot generate classification report.")


In [ ]:
val_data_with_anomalies.head()

In [ ]:
val_data_with_anomalies['anomaly'].value_counts()

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report

# Load the data with the first column as datetime and handle comma as decimal separator
file_path = 'LD2011_2014.txt'
data = pd.read_csv(
    file_path,
    sep=';',                        # Semicolon-separated
    header=0,                       # Use the first row as the header
    parse_dates=[0],                # Parse the first column as datetime
    index_col=0,                    # Set the first column as the index
    dayfirst=True,                  # Assume the date format is day-first
    decimal=','                     # Use comma as decimal separator
)

# Convert kW to kWh by dividing each value by 4
data_kwh = data / 4

# Force datetime conversion with 'errors=coerce' to identify problematic dates
data_kwh.index = pd.to_datetime(data_kwh.index, errors='coerce')

# Identify and display rows where the datetime index is invalid (NaT)
invalid_rows = data_kwh[data_kwh.index.isna()]
if not invalid_rows.empty:
    print("Rows with invalid datetime format:")
    print(invalid_rows)
    # Drop rows with NaT index
    data_kwh = data_kwh.dropna(subset=[data_kwh.index.name])

# Split the data by date to cover different seasons
train_data = data_kwh['2011-01-01':'2012-12-31']
val_data = data_kwh['2013-01-01':'2013-12-31']
test_data = data_kwh['2014-01-01':'2014-12-31']

# Print the shapes to verify
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

# Define a function to inject anomalies and label them
def inject_and_label_anomalies(data, anomaly_percentage=0.01, spike_factor=3, prolonged_anomaly_duration=4):
    # Ensure the index is in datetime format
    data.index = pd.to_datetime(data.index, errors='coerce').tz_localize(None)
    
    # Create a copy to avoid modifying the original data
    data_with_anomalies = data.copy()
    
    # Add a column for anomaly labels (0 = normal, 1 = anomaly)
    data_with_anomalies['anomaly'] = 0
    
    # Calculate the total number of anomalies to inject
    num_anomalies = int(len(data) * anomaly_percentage)
    
    # Inject short-term spikes and dips
    short_term_indices = np.random.choice(data.index.dropna(), size=num_anomalies, replace=False)
    for idx in short_term_indices:
        col = np.random.choice(data.columns)  # Choose a random column
        if np.random.rand() > 0.5:
            # Spike: Increase value
            data_with_anomalies.loc[idx, col] *= spike_factor
        else:
            # Dip: Decrease value
            data_with_anomalies.loc[idx, col] /= spike_factor
        # Mark as anomaly
        data_with_anomalies.loc[idx, 'anomaly'] = 1
    
    # Inject prolonged high consumption anomalies
    prolonged_indices = np.random.choice(data.index.dropna()[:-prolonged_anomaly_duration], size=num_anomalies // 2, replace=False)
    for idx in prolonged_indices:
        for i in range(prolonged_anomaly_duration):
            col = np.random.choice(data.columns)  # Choose a random column
            timestamp = idx + pd.Timedelta(minutes=15 * i)
            if timestamp in data_with_anomalies.index:
                data_with_anomalies.loc[timestamp, col] *= spike_factor
                data_with_anomalies.loc[timestamp, 'anomaly'] = 1
    
    return data_with_anomalies

# Inject anomalies into validation and test sets and label them
val_data_with_anomalies = inject_and_label_anomalies(val_data, anomaly_percentage=0.01, spike_factor=3)
test_data_with_anomalies = inject_and_label_anomalies(test_data, anomaly_percentage=0.01, spike_factor=3)

# Display sample rows with anomalies for verification
print("Sample validation data with anomalies:")
print(val_data_with_anomalies[val_data_with_anomalies['anomaly'] == 1].sample(5))

# Separate 'anomaly' columns for future evaluation if they exist
train_anomaly = train_data['anomaly'].copy() if 'anomaly' in train_data.columns else None
val_anomaly = val_data_with_anomalies['anomaly'].copy() if 'anomaly' in val_data_with_anomalies.columns else None

# Drop 'anomaly' column temporarily for model input
X_train = train_data.drop(columns=['anomaly'], errors='ignore')
X_val = val_data_with_anomalies.drop(columns=['anomaly'], errors='ignore')

# Train the Isolation Forest on X_train
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Predict anomalies on the validation set
# Store predictions in a new DataFrame to avoid overwriting val_data_with_anomalies
predictions = pd.DataFrame(index=val_data_with_anomalies.index)
predictions['predicted_anomaly'] = iso_forest.predict(X_val)
predictions['predicted_anomaly'] = predictions['predicted_anomaly'].apply(lambda x: 1 if x == -1 else 0)

# Re-attach original 'anomaly' column and predictions for evaluation
val_data_with_anomalies['anomaly'] = val_anomaly
val_data_with_anomalies['predicted_anomaly'] = predictions['predicted_anomaly']

# Fill NaN values in 'anomaly' and 'predicted_anomaly' columns if they exist
val_data_with_anomalies['anomaly'] = val_data_with_anomalies['anomaly'].fillna(0)
val_data_with_anomalies['predicted_anomaly'] = val_data_with_anomalies['predicted_anomaly'].fillna(0)

# Display a sample of results to confirm NaNs are handled
print("Sample of validation data with anomalies and predicted anomalies after filling NaNs:")
print(val_data_with_anomalies[['anomaly', 'predicted_anomaly']].sample(10))

# Evaluate Model Performance
if 'anomaly' in val_data_with_anomalies.columns:
    print("Classification Report for Validation Data:")
    print(classification_report(val_data_with_anomalies['anomaly'], val_data_with_anomalies['predicted_anomaly']))
else:
    print("'anomaly' column missing in validation data. Cannot generate classification report.")


Training data shape: (70175, 370)
Validation data shape: (35040, 370)
Test data shape: (35040, 370)
Sample validation data with anomalies:
                       MT_001    MT_002    MT_003     MT_004     MT_005  \
2013-09-03 13:45:00  3.489848  8.357041  0.434405  24.390244  10.670732   
2013-03-15 01:45:00  0.634518  5.689900  0.651607  22.865854  11.280488   
2013-12-07 04:00:00  0.634518  5.512091  0.434405  24.898374  14.634146   
2013-08-27 12:00:00  0.317259  8.001422  0.434405  30.995935  14.024390   
2013-12-29 23:30:00  0.634518  6.223329  0.434405  43.699187  21.036585   

                        MT_006    MT_007     MT_008     MT_009     MT_010  \
2013-09-03 13:45:00  49.107143  3.533070  83.333333  23.164336  15.322581   
2013-03-15 01:45:00  33.482143  1.130582  44.612795   9.615385  15.591398   
2013-12-07 04:00:00  36.458333  0.706614  51.346801  11.363636   9.139785   
2013-08-27 12:00:00  47.619048  5.652911  77.441077   8.304196   9.946237   
2013-12-29 23:30:00  68.4

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Load and preprocess the data
file_path = 'LD2011_2014.txt'
data = pd.read_csv(
    file_path,
    sep=';',                        
    header=0,                       
    parse_dates=[0],                
    index_col=0,                    
    dayfirst=True,                  
    decimal=','                     
)
data_kwh = data / 4

# Split the data
train_data = data_kwh['2011-01-01':'2012-12-31']
val_data = data_kwh['2013-01-01':'2013-12-31']
test_data = data_kwh['2014-01-01':'2014-12-31']

# Normalize with MinMaxScaler
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)
test_data = scaler.transform(test_data)

# Inject Anomalies for Validation and Test Sets
def inject_anomalies(data, anomaly_percentage=0.05, spike_factor=3, prolonged_duration=6):
    data_with_anomalies = data.copy()
    n_anomalies = int(anomaly_percentage * len(data))
    anomaly_indices = np.random.choice(len(data), n_anomalies, replace=False)

    for idx in anomaly_indices:
        col = np.random.randint(0, data.shape[1])  
        if np.random.rand() < 0.5:
            data_with_anomalies[idx, col] *= spike_factor  
        else:
            data_with_anomalies[idx, col] /= spike_factor  

        if np.random.rand() < 0.3:
            for i in range(prolonged_duration):
                if idx + i < len(data_with_anomalies):
                    data_with_anomalies[idx + i, col] *= spike_factor

    return data_with_anomalies, anomaly_indices

val_data_with_anomalies, val_anomaly_indices = inject_anomalies(val_data, anomaly_percentage=0.05)
test_data_with_anomalies, test_anomaly_indices = inject_anomalies(test_data, anomaly_percentage=0.05)

# Define a deeper Autoencoder model
input_dim = train_data.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
encoder = Dense(16, activation="relu")(encoder)
encoder = Dense(8, activation="relu")(encoder)
decoder = Dense(16, activation="relu")(encoder)
decoder = Dense(encoding_dim, activation="relu")(decoder)
decoder = Dense(input_dim, activation="sigmoid")(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer=Adam(learning_rate=0.0001), loss="mse")

# Train with reduced batch size
history = autoencoder.fit(train_data, train_data,
                          epochs=100,
                          batch_size=64,
                          shuffle=True,
                          validation_data=(val_data, val_data),
                          verbose=1)

# Set a threshold using validation data MSE
reconstructions = autoencoder.predict(val_data)
mse = np.mean(np.power(val_data - reconstructions, 2), axis=1)
threshold = np.percentile(mse, 90)

# Detect Anomalies
# Validation Set
reconstructions = autoencoder.predict(val_data_with_anomalies)
mse = np.mean(np.power(val_data_with_anomalies - reconstructions, 2), axis=1)
predicted_anomalies_val = mse > threshold

# Test Set
reconstructions = autoencoder.predict(test_data_with_anomalies)
mse = np.mean(np.power(test_data_with_anomalies - reconstructions, 2), axis=1)
predicted_anomalies_test = mse > threshold

# Evaluate results
val_true_labels = np.zeros(len(val_data_with_anomalies))
val_true_labels[val_anomaly_indices] = 1
print("Validation Set Classification Report:")
print(classification_report(val_true_labels, predicted_anomalies_val))

test_true_labels = np.zeros(len(test_data_with_anomalies))
test_true_labels[test_anomaly_indices] = 1
print("Test Set Classification Report:")
print(classification_report(test_true_labels, predicted_anomalies_test))


Epoch 1/100
1095/1095 [==============================] - 6s 5ms/step - loss: 0.0413 - val_loss: 54631.5000
Epoch 2/100
1095/1095 [==============================] - 5s 5ms/step - loss: 0.0087 - val_loss: 54631.4805
Epoch 3/100
1095/1095 [==============================] - 6s 5ms/step - loss: 0.0064 - val_loss: 54631.4805
Epoch 4/100
1095/1095 [==============================] - 6s 5ms/step - loss: 0.0061 - val_loss: 54631.4844
Epoch 5/100
1095/1095 [==============================] - 6s 5ms/step - loss: 0.0059 - val_loss: 54631.4844
Epoch 6/100
1095/1095 [==============================] - 6s 6ms/step - loss: 0.0057 - val_loss: 54631.4844
Epoch 7/100
1095/1095 [==============================] - 6s 6ms/step - loss: 0.0056 - val_loss: 54631.4883
Epoch 8/100
1095/1095 [==============================] - 6s 6ms/step - loss: 0.0054 - val_loss: 54631.4844
Epoch 9/100
1095/1095 [==============================] - 6s 6ms/step - loss: 0.0053 - val_loss: 54631.4844
Epoch 10/100
1095/1095 [=============

In [ ]:
'''
50 epochs
Validation Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.90      0.92     33197
         1.0       0.05      0.11      0.07      1747

    accuracy                           0.86     34944
   macro avg       0.50      0.50      0.50     34944
weighted avg       0.91      0.86      0.88     34944

Test Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.90      0.92     33197
         1.0       0.05      0.10      0.07      1747

    accuracy                           0.86     34944
   macro avg       0.50      0.50      0.50     34944
weighted avg       0.91      0.86      0.88     34944

100 epochs
Validation Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.90      0.92     33197
         1.0       0.05      0.09      0.06      1747

    accuracy                           0.86     34944
   macro avg       0.50      0.50      0.49     34944
weighted avg       0.90      0.86      0.88     34944

Test Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.90      0.92     33197
         1.0       0.05      0.10      0.07      1747

    accuracy                           0.86     34944
   macro avg       0.50      0.50      0.50     34944
weighted avg       0.91      0.86      0.88     34944

#LSTM v1
Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     31450
           1       0.10      0.05      0.07      3494

    accuracy                           0.86     34944
   macro avg       0.50      0.50      0.50     34944
weighted avg       0.82      0.86      0.84     34944

Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93     31450
           1       0.11      0.04      0.06      3494

    accuracy                           0.87     34944
   macro avg       0.51      0.50      0.50     34944
weighted avg       0.82      0.87      0.84     34944
'''

In [ ]:
#LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load and prepare data
file_path = 'LD2011_2014.txt'
data = pd.read_csv(
    file_path,
    sep=';',
    parse_dates=[0],
    index_col=0,
    dayfirst=True,
    decimal=','
)
data_kwh = data / 4  # Convert to kWh

# Ensure the index is a datetime format for all datasets
data_kwh.index = pd.to_datetime(data_kwh.index)

# Split data
train_data = data_kwh['2011-01-01':'2012-12-31']
val_data = data_kwh['2013-01-01':'2013-12-31']
test_data = data_kwh['2014-01-01':'2014-12-31']

# Anomaly Injection
def inject_and_label_anomalies(data, anomaly_percentage=0.15, spike_factor=1.5, prolonged_anomaly_duration=4):
    np.random.seed(42)
    data_with_anomalies = data.copy()
    data_with_anomalies['anomaly'] = 0
    num_anomalies = int(len(data) * anomaly_percentage)

    # Inject sudden spikes and dips
    for _ in range(num_anomalies):
        idx = np.random.choice(data.index)
        col = np.random.choice(data.columns)
        if np.random.rand() > 0.5:
            data_with_anomalies.at[idx, col] *= spike_factor
        else:
            data_with_anomalies.at[idx, col] /= spike_factor
        data_with_anomalies.at[idx, 'anomaly'] = 1

    # Inject prolonged anomalies
    prolonged_indices = np.random.choice(data.index[:-prolonged_anomaly_duration], num_anomalies // 2, replace=False)
    for idx in prolonged_indices:
        if not isinstance(idx, pd.Timestamp):  # Ensure index is a Timestamp
            continue
        col = np.random.choice(data.columns)
        for i in range(prolonged_anomaly_duration):
            data_with_anomalies.at[idx + pd.Timedelta(minutes=15 * i), col] *= spike_factor
            data_with_anomalies.at[idx + pd.Timedelta(minutes=15 * i), 'anomaly'] = 1

    return data_with_anomalies

# Inject anomalies
val_data_with_anomalies = inject_and_label_anomalies(val_data)
test_data_with_anomalies = inject_and_label_anomalies(test_data)

# Scale data
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(np.clip(train_data, -1e10, 1e10))  # Clip extreme values
val_data_scaled = scaler.transform(np.clip(val_data_with_anomalies.drop(columns=['anomaly']), -1e10, 1e10))
test_data_scaled = scaler.transform(np.clip(test_data_with_anomalies.drop(columns=['anomaly']), -1e10, 1e10))

# Prepare data for LSTM input format (3D array)
def create_sequences(data, seq_length=48):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        sequences.append(seq)
    return np.array(sequences)

X_train = create_sequences(train_data_scaled)
X_val = create_sequences(val_data_scaled)
X_test = create_sequences(test_data_scaled)

# Reshape for anomaly labels
y_val = val_data_with_anomalies['anomaly'][48:].values  # Shift by seq_length
y_test = test_data_with_anomalies['anomaly'][48:].values

# Check for NaNs in the dataset before feeding to the model
assert not np.isnan(X_train).any(), "NaNs detected in X_train"
assert not np.isnan(X_val).any(), "NaNs detected in X_val"
assert not np.isnan(X_test).any(), "NaNs detected in X_test"

# LSTM Autoencoder Model with Gradient Clipping
model = Sequential([
    LSTM(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(64, activation='relu', return_sequences=False),
    RepeatVector(X_train.shape[1]),
    LSTM(64, activation='relu', return_sequences=True),
    Dropout(0.3),
    LSTM(128, activation='relu', return_sequences=True),
    TimeDistributed(Dense(X_train.shape[2]))
])

optimizer = Adam(learning_rate=0.0005, clipnorm=1.0)  # Reduced learning rate with gradient clipping
model.compile(optimizer=optimizer, loss='mse')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train Model
history = model.fit(
    X_train, X_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, X_val),
    callbacks=[early_stopping],
    verbose=1
)

# Calculate Reconstruction Loss Threshold
reconstructions = model.predict(X_train)
train_loss = np.mean(np.square(reconstructions - X_train), axis=(1, 2))
threshold = np.percentile(train_loss, 95)

# Predict Anomalies on Validation and Test Sets
def detect_anomalies(model, data, threshold):
    reconstructions = model.predict(data)
    loss = np.mean(np.square(reconstructions - data), axis=(1, 2))
    return (loss > threshold).astype(int), loss

val_preds, val_loss = detect_anomalies(model, X_val, threshold)
test_preds, test_loss = detect_anomalies(model, X_test, threshold)

# Evaluation
print("Validation Set Classification Report:")
print(classification_report(y_val, val_preds))

print("Test Set Classification Report:")
print(classification_report(y_test, test_preds))



Epoch 1/50
1096/1096 [==============================] - 781s 700ms/step - loss: 0.0474 - val_loss: 1554628736.0000
Epoch 2/50
1096/1096 [==============================] - 706s 645ms/step - loss: 0.0047 - val_loss: 113374720.0000
Epoch 3/50
1096/1096 [==============================] - 725s 661ms/step - loss: 0.0038 - val_loss: 65938828.0000
Epoch 4/50
1096/1096 [==============================] - 691s 630ms/step - loss: 0.0033 - val_loss: 179972653056.0000
Epoch 5/50
1096/1096 [==============================] - 690s 629ms/step - loss: 0.0030 - val_loss: 2654580736.0000
Epoch 6/50
1096/1096 [==============================] - 688s 627ms/step - loss: 0.0028 - val_loss: 8348533.0000
Epoch 7/50
1096/1096 [==============================] - 727s 663ms/step - loss: 0.0026 - val_loss: 15705239.0000
Epoch 8/50
1096/1096 [==============================] - 703s 642ms/step - loss: 0.0025 - val_loss: 196708768.0000
Epoch 9/50
1096/1096 [==============================] - 674s 615ms/step - loss: 0.0024 

KeyboardInterrupt: 

In [19]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Parameters for synthetic data generation
num_meters = 5
num_days = 5 * 365  # 5 years of data
sampling_rate = 15  # in minutes
anomaly_percentage = 0.1  # 10% anomalies

# Calculate total samples based on days and sampling rate
total_samples = num_days * 24 * (60 // sampling_rate)
time_index = pd.date_range(start="2010-01-01", periods=total_samples, freq=f"{sampling_rate}T")

# Initialize DataFrame with time index
data = pd.DataFrame(index=time_index)

# Generate baseline consumption patterns
for meter_id in range(1, num_meters + 1):
    daily_pattern = 0.5 + 0.2 * np.sin(np.linspace(0, 2 * np.pi, 24 * (60 // sampling_rate))) * (1 + np.random.normal(0, 0.01))
    weekly_pattern = 0.5 + 0.1 * np.sin(np.linspace(0, 2 * np.pi, 7 * 24 * (60 // sampling_rate))) * (1 + np.random.normal(0, 0.01))
    
    daily_pattern_repeated = np.tile(daily_pattern, num_days)
    weekly_pattern_repeated = np.tile(weekly_pattern, num_days // 7 + 1)[:total_samples]
    
    baseline_consumption = daily_pattern_repeated + weekly_pattern_repeated[:total_samples] + np.random.normal(0, 0.05, total_samples)
    data[f'meter_{meter_id}'] = baseline_consumption

# Function to inject anomalies
def inject_anomalies(data, anomaly_percentage, spike_factor=3, dip_factor=0.3, prolonged_anomaly_duration=4):
    num_anomalies = int(len(data) * anomaly_percentage)
    anomaly_indices = np.random.choice(data.index, size=num_anomalies, replace=False)
    
    data['anomaly'] = 0  # Column to track anomalies
    
    for idx in anomaly_indices:
        col = np.random.choice(data.columns[:-1])  # Random meter column
        
        if np.random.rand() > 0.5:
            data.loc[idx, col] *= spike_factor  # Sudden spike
        else:
            data.loc[idx, col] *= dip_factor  # Sudden dip
            
        # Prolonged anomalies
        for i in range(prolonged_anomaly_duration):
            future_idx = idx + pd.Timedelta(minutes=sampling_rate * i)
            if future_idx in data.index:
                data.loc[future_idx, col] *= spike_factor if np.random.rand() > 0.5 else dip_factor
                data.loc[future_idx, 'anomaly'] = 1

    return data

# Apply anomaly injection
data_with_anomalies = inject_anomalies(data.copy(), anomaly_percentage)

# Data Scaling
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_with_anomalies.drop(columns=['anomaly']))

# Convert data into supervised learning format for LSTM
def create_sequences(data, seq_length=48):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data_with_anomalies['anomaly'].iloc[i + seq_length])  # Label is the next point
    return np.array(X), np.array(y)

seq_length = 48  # 12 hours of 15-minute intervals
X, y = create_sequences(scaled_data, seq_length=seq_length)

# Split into train, validation, and test sets
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

# LSTM Autoencoder Model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(seq_length, X.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='relu', return_sequences=False),
    RepeatVector(seq_length),
    LSTM(32, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(64, activation='relu', return_sequences=True),
    TimeDistributed(Dense(X.shape[2]))
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,  # Number of epochs to wait before stopping if no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

checkpoint = ModelCheckpoint(
    filepath='best_model.h5',  # Path to save the best model
    monitor='val_loss', 
    save_best_only=True, 
    verbose=1
)

# Add the callbacks to your fit function
history = model.fit(
    X_train, X_train,
    epochs=50,  # Adjust as needed
    batch_size=128,
    validation_data=(X_val, X_val),
    callbacks=[early_stopping, checkpoint]
)

# Set threshold for anomaly detection
X_train_pred = model.predict(X_train)
train_mae_loss = np.mean(np.abs(X_train_pred - X_train), axis=(1, 2))
threshold = np.percentile(train_mae_loss, 95)

# Evaluate on test set
X_test_pred = model.predict(X_test)
test_mae_loss = np.mean(np.abs(X_test_pred - X_test), axis=(1, 2))
y_test_pred = (test_mae_loss > threshold).astype(int)

# Classification report
print("Test Set Classification Report:")
print(classification_report(y_test, y_test_pred))

Epoch 1/50
958/958 [==============================] - ETA: 0s - loss: 0.0053
Epoch 1: val_loss improved from inf to 0.00446, saving model to best_model.h5
958/958 [==============================] - 610s 630ms/step - loss: 0.0053 - val_loss: 0.0045
Epoch 2/50
958/958 [==============================] - ETA: 0s - loss: 0.0042
Epoch 2: val_loss improved from 0.00446 to 0.00443, saving model to best_model.h5
958/958 [==============================] - 586s 611ms/step - loss: 0.0042 - val_loss: 0.0044
Epoch 3/50
958/958 [==============================] - ETA: 0s - loss: 0.0041
Epoch 3: val_loss improved from 0.00443 to 0.00440, saving model to best_model.h5
958/958 [==============================] - 601s 627ms/step - loss: 0.0041 - val_loss: 0.0044
Epoch 4/50
958/958 [==============================] - ETA: 0s - loss: 0.0041
Epoch 4: val_loss improved from 0.00440 to 0.00428, saving model to best_model.h5
958/958 [==============================] - 589s 615ms/step - loss: 0.0041 - val_loss: 0.0